# 다중 PDF 파일을 활용한 RAG 파이프라인 구현


이 노트북은 다음과 같은 기능을 가진 Retrieval-Augmented Generation (RAG) 파이프라인을 구현
- 여러 PDF 파일을 효율적으로 처리
- Pinecone을 이용한 벡터 데이터베이스 설정 및 Upstage Solar Embedding 활용
- 근데 일단 Faiss
- 하이퍼파라미터 튜닝이 적용된 검색 및 선택적 재정렬
- Upstage Solar Pro를 이용한 답변 생성 및 Upstage Groundness Check API를 통한 평가


In [ ]:

# 필요한 라이브러리 임포트
from dotenv import load_dotenv
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain_upstage import UpstageEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from pinecone import Pinecone, ServerlessSpec
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain_upstage import ChatUpstage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParse

import pinecone
import os

# Pinecone 초기화
load_dotenv()

ImportError: cannot import name 'GroundnessCheckAPI' from 'langchain.evaluation' (c:\Users\SSAFY\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain\evaluation\__init__.py)

In [ ]:

# Pinecone 클라이언트 설정
pc = Pinecone(
    api_key=os.getenv("PINECONE_API_KEY"),
)


# Pinecone 인덱스 생성
index_name = "document-index"
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric="euclidean",
        spec=ServerlessSpec(
            cloud="aws",
            region=os.getenv("PINECONE_ENVIRONMENT"),
        ),
    )



In [ ]:

# 여러 PDF 파일을 처리하고 텍스트를 청크로 나누는 함수
def process_pdfs(pdf_files, chunk_size=1000, chunk_overlap=100):
    all_documents = []
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    
    for pdf_file in pdf_files:
        loader = PyPDFLoader(pdf_file)
        documents = loader.load()
        texts = splitter.split_documents(documents)
        all_documents.extend(texts)
    
    return all_documents

# 예제: 여러 PDF 파일 처리
pdf_files = ["Hyundai_NewYear.pdf", "LG_NewYear.pdf", "Samsung_NewYear.pdf", "SK_NewYear.pdf", "CJ_NewYear.pdf"]
documents = process_pdfs(pdf_files)


In [ ]:

# 임베딩 및 벡터 DB 설정
embedding_model = UpstageEmbeddings(model="solar-embedding-1-large")
index_name = "new-year-speeches"

if index_name not in pinecone.list_indexes():
    pinecone.create_index(index_name, dimension=embedding_model.dimension)

vector_db = Pinecone(index=pinecone.Index(index_name), embedding_function=embedding_model.embed)

# 벡터 데이터베이스에 문서 추가
vector_db.add_texts(documents)


In [ ]:

# 검색기 설정 (하이퍼파라미터 튜닝 포함)
retriever = vector_db.as_retriever(search_type="similarity", search_kwargs={"k": 4, "similarity_threshold": 0.7})

# 선택적 재정렬기 추가
from langchain.retrievers import CrossEncoderReranker
reranker = CrossEncoderReranker(retriever=retriever, rerank_top_k=10)


In [ ]:

# Upstage Solar Pro를 이용한 QA 체인 설정
llm = UpstageSolar()
qa_chain = RetrievalQA(llm=llm, retriever=retriever)

# 예제 질문
query = "2025년 신년사에서 제시된 주요 목표는 무엇인가요?"
response = qa_chain.run(query)

# Groundness Check API를 이용한 응답 평가
evaluator = GroundnessCheckAPI()
evaluation = evaluator.evaluate(response)
print("생성된 응답:", response)
print("평가 결과:", evaluation)



## 요약
이 노트북은 여러 PDF 파일을 처리하고, 효율적인 검색 및 평가 기능을 제공하는 완전한 RAG 파이프라인을 구현합니다. API 키와 파일 경로를 실제 값으로 대체하여 실행해보세요.
